<h2> Peak-Gene <h2>

In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
import pybedtools
import SpaDC

# 1
tss = pd.read_csv('/home/nas2/biod/machuanlong/data/GRN/mm10_TSS.txt', sep='\t')
print(tss)

tss['end'] = tss['start'] + 250000
tss['start'] = tss['start'] - 250000

print(tss[tss['start'] < 0])
tss.loc[tss['start'] < 0, 'start'] = 0
print(tss[tss['start'] == 0])

tss.columns = [0, 1, 2, 'gene']
print(tss)

tss.to_csv('/home/nas2/biod/machuanlong/data/GRN/mm10_TSS.bed', sep='\t', index=False)

In [ ]:
# 2
rna = sc.read_h5ad('/home/nas2/biod/machuanlong/data/MISAR_seq/MISAR_seq_mouse_E15_brain_mRNA_data.h5ad')
result = sc.read_h5ad('/home/nas2/biod/machuanlong/result/misar_result.h5ad')

rna.obs['SpaDC'] = result.obs['SpaDC'].values

sc.pp.filter_cells(rna, min_genes=200)
sc.pp.filter_genes(rna, min_cells=3)

rna.var['mt'] = rna.var_names.str.startswith('mt-')
sc.pp.calculate_qc_metrics(rna, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
rna = rna[rna.obs['pct_counts_mt'] < 10, :]
print(rna)

sc.pp.normalize_total(rna, target_sum=1e4)
sc.pp.log1p(rna)

sc.tl.rank_genes_groups(rna, groupby='SpaDC', method='wilcoxon')
marker_genes = (
    sc.get.rank_genes_groups_df(rna, group=None)
    .groupby('group')['names']
    .apply(lambda x: x.head(25))
    .explode()
    .unique()
    .tolist()
)

tss = pd.read_csv('/home/nas2/biod/machuanlong/data/GRN/mm10_TSS.bed', sep='\t')

tss = tss[tss['gene'].isin(marker_genes)]

tss = tss.drop_duplicates('gene')

tss.to_csv('/home/nas2/biod/machuanlong/data/GRN/misar/marker_genes_TSS.bed',sep='\t', index=False)

In [ ]:
# 3
atac = sc.read_h5ad('/home/nas2/biod/machuanlong/data/MISAR_seq/MISAR_seq_mouse_E15_brain_ATAC_data.h5ad')
index = atac.var_names
index = pd.DataFrame(x.split('-') for x in index)
index.to_csv('/home/nas2/biod/machuanlong/data/GRN/misar/ATAC.bed', sep='\t', header=False, index=False)

ATAC = pybedtools.BedTool('/home/nas2/biod/machuanlong/data/GRN/misar/ATAC.bed')
tss = pybedtools.BedTool('/home/nas2/biod/machuanlong/data/GRN/misar/marker_genes_TSS.bed')  

overlap = tss.intersect(ATAC, wo=True)
overlap.moveto('/home/nas2/biod/machuanlong/data/GRN/misar/overlap.bed')

In [ ]:
# 4
overlap = pd.read_csv('/home/nas2/biod/machuanlong/data/GRN/misar/overlap.bed', sep='\t', header=None)
print(overlap)

# get overlap_peak.bed 
overlap.iloc[:, 4:7].drop_duplicates().to_csv('/home/nas2/biod/machuanlong/data/GRN/misar/overlap_peak.bed', sep='\t', index=False, header=None)

group = overlap.groupby([3])

gene_list = []
peak_list = []

for x, y in group:
    gene_list.append(x[0])
    peak = []
    for i in range(len(y)):
        peak.append('-'.join('%s'%a for a in y.iloc[i, 4:7]))
    peak_list.append(peak)
    
rna = sc.read_h5ad('/home/nas2/biod/machuanlong/data/MISAR_seq/MISAR_seq_mouse_E15_brain_mRNA_data.h5ad')
rna.X = rna.X.todense()

# get atac_denoise 
atac = sc.read_h5ad('/home/nas2/biod/machuanlong/data/MISAR_seq/MISAR_seq_mouse_E15_brain_ATAC_data.h5ad')
seq = pd.read_csv('/home/nas2/biod/machuanlong/data/MISAR_seq/seqs.txt', sep='\t')

atac_denoise = SpaDC.get_denoise_adata(atac, seq, '/home/nas2/biod/machuanlong/result/misar.pt')
atac.X = atac_denoise.X

gene_index =  list(rna.var_names.get_indexer(gene_list))
peak_index = []
for peak in peak_list:
    index = list(atac.var_names.get_indexer(peak))
    peak_index.append(index)

import xgboost as xgb
import matplotlib.pyplot as plt
import shap

index = list(atac.obs_names)

all_df = pd.DataFrame()
top_peaks_list = []
for i in range(len(gene_index)):
    X = atac[:, peak_index[i]].X
    y = rna[:, gene_index[i]].X

    model = xgb.XGBRegressor()
    model.fit(X,y)
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)

    # top 6 peak，+,-
    mean_shap = shap_values.mean(axis=0) # shape: (n_peaks,)
    top_indices = np.abs(mean_shap).argsort()[::-1][:6]
    top_peaks = np.array(peak_list[i])[top_indices]
    top_shap = mean_shap[top_indices]

    max_abs = np.max(np.abs(top_shap))
    if max_abs == 0:
        norm_shap = np.zeros_like(top_shap)
    else:
        norm_shap = top_shap / max_abs

    top_df = pd.DataFrame({
        'gene': gene_list[i],
        'peak': top_peaks,
        'mean_shap': top_shap,
        'norm_shap': norm_shap
    })

    all_df = pd.concat([all_df, top_df], ignore_index=True)

    top_peaks_list.extend(top_peaks)

all_df.to_csv("/home/nas2/biod/machuanlong/data/GRN/misar/result/gene_top_peaks.csv", index=False)

unique_peaks = sorted(set(top_peaks_list)) 
top_peaks_df = pd.DataFrame(x.split('-') for x in unique_peaks)
print(top_peaks_df)
top_peaks_df.to_csv('/home/nas2/biod/machuanlong/data/GRN/misar/result/all_top_peaks.bed', sep='\t', index=False, header=None)


<h2> TF-Peak <h2>

In [ ]:
import scanpy as sc
from pathlib import Path
import json
import torch
from scipy.stats import norm
import SpaDC

adata = sc.read_h5ad('home/nas2/biod/machuanlong/data/MISAR_seq/MISAR_seq_mouse_E15_brain_ATAC_data.h5ad')
print(adata)

model = SpaDC.SpaDC(adata.X.shape[0])
model.load_state_dict(torch.load('home/nas2/biod/machuanlong/result/misar.pt'))

# get d_background
folder = Path("home/nas2/biod/machuanlong/data/GRN/misar/Mus_musculus_motif_fasta/shuffled_peaks_motifs")
fasta_files = list(folder.glob("*.fasta"))

fasta_bg = "home/nas2/biod/machuanlong/data/GRN/misar/Mus_musculus_motif_fasta/shuffled_peaks.fasta"

d_background_dict = {}

pred_bg_mean = SpaDC.pred_on_fasta(fasta_bg, model).mean(axis=0)
for fasta_motif in fasta_files:
    pred_motif_mean = SpaDC.pred_on_fasta(fasta_motif, model).mean(axis=0)
    d_background = pred_motif_mean - pred_bg_mean
    d_background_dict[fasta_motif.stem] = d_background.tolist()

with open("/home/nas2/biod/machuanlong/data/GRN/misar/result/d_background.json", "w") as f:
    json.dump(d_background_dict, f)

# get peak_tf_activity
with open("/home/nas2/biod/machuanlong/data/GRN/misar/result/d_background.json", "r") as f:
    d_background_dict = json.load(f)

folder = Path("/home/nas2/biod/machuanlong/data/GRN/misar/peak_motif_perturbation")

result = []
for subfolder in folder.iterdir():  
    pred_peak = SpaDC.pred_on_fasta("%s/original.fasta" % subfolder, model)
    for fasta_file in subfolder.glob("*.fasta"):
        if fasta_file.name != "original.fasta":                    
            pred_motif_Perturbation_mean = SpaDC.pred_on_fasta(fasta_file, model).mean(axis=0)
            d_pi = pred_peak - pred_motif_Perturbation_mean
            
            tf_name = str(fasta_file.stem).replace("_Perturbation", "")
            d_pi = np.array(d_pi)
            d_background = np.array(d_background_dict[tf_name])

            mu_bg = np.mean(d_background)
            std_bg = np.std(d_background)

            z_scores = (d_pi - mu_bg) / std_bg

            p_values = 1 - norm.cdf(z_scores)

            activity = np.mean(1 - p_values)

            result.append([subfolder.name, tf_name, activity])
    
df = pd.DataFrame(result, columns=["peak", "tf", "activity"])

df.to_csv("/home/nas2/biod/machuanlong/data/GRN/misar/result/peak_tf_activity.csv", index=False)

<h2> TF-Gene <h2>

In [ ]:
import pandas as pd

gene_top_peaks = pd.read_csv("/home/nas2/biod/machuanlong/data/GRN/misar/result/gene_top_peaks.csv")  
peak_tf_activity = pd.read_csv("/home/nas2/biod/machuanlong/data/GRN/misar/result/peak_tf_activity.csv")  

merged = pd.merge(gene_top_peaks, peak_tf_activity, on="peak")
print(merged)

threshold = 0.8
merged = merged[merged["activity"] > threshold]
print(merged)

# activity × norm_shap
merged["weighted_activity"] = merged["activity"] * merged["norm_shap"]
result = merged.groupby(["tf", "gene"])["weighted_activity"].mean().reset_index()

result.rename(columns={"weighted_activity": "activity"}, inplace=True)
print(result)

result.to_csv("/home/nas2/biod/machuanlong/data/GRN/misar/result/tf_gene_activity.csv", index=False)

